# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This type of problem could be approached as either a classification problem or a regression problem. Classification problems try to predict a discrete value. In this case, we would be trying to predict whether a student falls into a pass group or a fail group. Regression problems look to predict a continuous value. With this type of problem, we would try to predict a student’s grade, and then see if that grade is high enough to pass or not. 

Looking at the data available for this project, I can see that the last column states whether a student passes or not. It does not give a grade. Therefore, I will be taking a classification approach to this project. Since I only have pass or fail information to work with, I can only predict a pass or fail value for each student. Classification is the proper choice for this case.


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [39]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [40]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/(float(n_passed) + float(n_failed)))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [41]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [42]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [43]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train =300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=1)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** Since this is a classification problem, I chose three supervised learning models that are better suited for predicting what group a new, unseen input would fall into. The three models that I chose are: the Support Vector Classifier (SVC), Artificial Neural Network (ANN), and the Decision Tree Classifier. These are very strong classification models that can predict discrete values for test data.

The Support Vector Classifier (SVC) is a form of the broader Support Vector Machine (SVM). These are used in finance to do things like make predictions on the price movements of certain stocks or indexes. They are also used in mechanically-oriented industries  to make predictions on what pieces of machinery are going to fail or need maintenance (Kimotho et al. 619-620).

SVMs work well with complicated data domains having high dimensional spaces. This model can also be effective in cases where the number of features is greater than the number of samples. SVMs are versatile, as they can employ different Kernel functions, including custom kernels. SVMs are not well suited for data sets that are very large, as the training time has a more than quadratic relation to the size of the data set ("1.4. Support Vector Machines — Scikit-Learn 0.17.1 Documentation").

It is for these reasons that I chose this model for this dataset. There are a large number of features in the student dataset relative to the number of samples, but the number of samples is not so great that it will overwhelm the SVC.



The Artificial Neural Network (ANN) is modeled after biological neurons that operate in a web-like, interlinked structure. Since they are modeled after biological neural systems, ANNs are used in various medical fields to gain better understanding of how those biological systems work (Strickland). The perceptron is one of several different elements that can be used in building an ANN, and is the one used in this project.

Artificial Neural Networks are extremely powerful, as they can model very complex relationships between features. They are able to learn from the data itself. Because of this, less understanding is needed of the underlying data. Downsides of ANNs are that they can take a long time to train and require significant processing power for large datasets. Also, the trained model is not easy to understand (Hamilton). 

I chose to use an ANN because the student dataset has many features that may have complex relationships with one another. The dataset is not too large, and so will not overwhelm this kind of model. However, since this model is so computationally intensive, it may not be the best choice if it must be run often or quickly to satisfy the needs of the client. 


Decision Trees are used in many fields such as finance, marketing, engineering, and medicine. In finance, for example, a mortgage company may use Decision Trees to classify a potential customer as being risky or non-risky in terms of being able to pay back the mortgage on time (Rokach and Maimon 9-11).

Decision Trees and the results they produce are simple to understand conceptually. It is easier to graphically represent the results of a Decision Tree than it is to represent the results of most other classifiers, such as SVMs. Since they are relatively simple, it is possible to build more complex and powerful classifiers using Decision Trees.

A downside to Decision Trees is that they can be prone to overfitting the data, especially if a lot of features are involved. This can be combated somewhat by tuning the parameters of the tree carefully. 

I would consider this model for this kind of application because it is simple. Its simplicity would allow us to help explain and graphically show the problem to the board of directors, making it easier for them to understand. Decision Trees can also help to extract relationships between certain features. This in itself is useful, to help better understand the underlying features that may influence a student to graduate or not. 

Sources:

Kimotho, James K. et al. CHEMICAL ENGINEERING TRANSACTIONS. Italy: AIDIC Servizi S.r.l., 2013. Print.
"1.4. Support Vector Machines — Scikit-Learn 0.17.1 Documentation". Scikit-learn.org. N.p., 2016. Web. 07 July 2016. 

Strickland, Jeffrey. Predictive Analytics Using R. N.p.: Lulu, 2014. Print. 

Hamilton, Laura Diane. "Machine Learning Algorithm Cheat Sheet." Amazon Services LLC Associates Program, 09 Sept. 2014. Web. 12 July 2016.

Rokach, Lior and Oded Maimon. Data Mining With Decision Trees. Singapore: World Scientific Publishing Co., Pte. Ltd., 2008. Print.




### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [44]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label= 'yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [45]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import svm
# from sklearn import model_B
#from sklearn import linear_model
from sklearn.linear_model import perceptron
# from skearln import model_C
from sklearn import tree

# TODO: Initialize the three models
clf_A = svm.SVC(random_state = 2)
clf_B = perceptron.Perceptron(random_state = 2)
clf_C = tree.DecisionTreeClassifier(random_state = 2)
clf = [clf_A, clf_B, clf_C]

# TODO: Set up the training set sizes
#X_train_100 = None
#y_train_100 = None
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_all, y_all, 
                                                    test_size=float(X_all.shape[0]-100)/float(X_all.shape[0]), random_state=1)
#print X_train_100.shape

#X_train_200 = None
#y_train_200 = None
X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_all, y_all, 
                                                    test_size=float(X_all.shape[0]-200)/float(X_all.shape[0]), random_state=1)

#X_train_300 = None
#y_train_300 = None
X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_all, y_all, 
                                                    test_size=float(X_all.shape[0]-300)/float(X_all.shape[0]), random_state=1)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for classifier in clf:
    #print classifier
    train_predict(classifier, X_train_100, y_train_100, X_test_100, y_test_100)
    train_predict(classifier, X_train_200, y_train_200, X_test_200, y_test_200)
    train_predict(classifier, X_train_300, y_train_300, X_test_300, y_test_300)




Training a SVC using a training set size of 100. . .
Trained model in 0.0070 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8609.
Made predictions in 0.0060 seconds.
F1 score for test set: 0.8067.
Training a SVC using a training set size of 200. . .
Trained model in 0.0090 seconds
Made predictions in 0.0060 seconds.
F1 score for training set: 0.8608.
Made predictions in 0.0050 seconds.
F1 score for test set: 0.8205.
Training a SVC using a training set size of 300. . .
Trained model in 0.0180 seconds
Made predictions in 0.0130 seconds.
F1 score for training set: 0.8584.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8462.
Training a Perceptron using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8312.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8180.
Training a Perceptron using a training set size of 200. . .
Trained model in 0.0010 seconds
Made pred

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.



** Classifer 1 - SVC** (See note above)

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0180|0.0040|0.8609|0.8067|
| 200               |0.0070|0.0050|0.8608|0.8205|
| 300               |0.0150|0.0030|0.8584|0.8462|

** Classifer 2 - Perceptron** (See note above) 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0010|0.0010|0.8312|0.8180|
| 200               |0.0020|0.0000|0.7907|0.8013|
| 300               |0.0020|0.0010|0.8074|0.8481|

** Classifer 3 - DecisionTreeClassifier** (See note above) 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0010|0.0010|1.0000|0.7160|
| 200               |0.0080|0.0010|1.0000|0.7407|
| 300               |0.0050|0.0010|1.0000|0.6880|

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** According to the results listed above, the best model for the job would be the Support Vector Classifier. This is because it has the highest F1 scores. These scores take into account both the precision and the recall of the results produced by a model. The Perceptron also has high F1 scores, but the SVC has slightly better overall F1 scores on the training and test data.

The SVC model does well with a higher number of features and a lower number of sample points. In predicting what students will not graduate, we face a binary classification problem, which is what the SVC is best suited for. And since this dataset is small, the performance of the SVC is very quick; the model can run its predictions in under a second. This model can scale up to accommodate more students, so long as there aren’t millions more. Since all calculations are done so quickly, the money and resources that would be needed for this kind of model are very low. Because of these factors, the SVC is the best model for this application. 



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** In simple terms, this model makes predictions as to whether a student is expected graduate or not, based on certain characteristics, or features, of that student. These features include things such as: their age, what kind of job they have, how much time they have to study, their health, and many more. This model has been given real life data for a number of students that have made it to graduation, and for those who have failed to do so. 

The Support Vector Classifier is able to manipulate this data in such a way that it can separate with a plane two distinct groups; students that graduate, and those who don’t. This manipulation is done using mathematical functions called ‘kernels’. The SVC then maximizes the distance between the two groups, which is called the ‘margin’. By maximizing the margin, the SVC has a greater chance of properly classifying future students into one group or the other. 

We can use this model to take in the data of a new student, and see with high accuracy whether that student will graduate or not. It is important that we can help more students who are in danger of not graduating, and help them before it’s too late. This model will greatly help us find who those students are. 



### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [46]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit
# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear', 'rbf', 'sigmoid'), 'C':[10, 14, 18], 'gamma': [0.01, 0.001, 0.0001]}


# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
sss = StratifiedShuffleSplit(y_train, n_iter=10, test_size=0.24, random_state=2)
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring = f1_scorer, cv=sss)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print 'The best parameters were: ', grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

The best parameters were:  {'kernel': 'rbf', 'C': 14, 'gamma': 0.001}
Made predictions in 0.0090 seconds.
Tuned model has a training F1 score of 0.8266.
Made predictions in 0.0030 seconds.
Tuned model has a testing F1 score of 0.8609.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The final tuned model’s F1 scores are 0.8266 on the training data, and 0.8609 on the testing data. The testing score is slightly lower for the tuned model than it is un-tuned model. Due to the fact that our dataset has significantly more positive labels than negative ones, I used StratifiedShuffleSplit to ensure that negative labels exist in all of our folds during cross-validation. This is to better represent samples from a future, probably larger dataset.

However, the testing F1 score is higher for the tuned model than it is for the un-tuned one. This shows that our tuning by means of GridSearchCV has helped to improve the accuracy of our model on the testing set. We are more interested with the F1 score of our model classifying a testing set than we are with the score of it classifying a training set. Overall, we are happy with the improved results produced by GridSearchCV.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.